In [41]:
import pandas as pd
import json
import os
from tqdm import tqdm

# json list생성
path_dir = "./submissions/"
json_list = os.listdir(path_dir)
json_list

# json 불러서 summary data에 대한 dictionary 만들기
summary_dict = {}

for i, json_file in enumerate(json_list):
    if json_file.endswith('.json'):
        pass
    else:
        continue
    with open(path_dir+json_file,"r") as f:
        data = json.load(f)
        
    for j,d in enumerate(tqdm(data, desc='data: '+str(i))):
        try:
            summary_dict[j].append(d['summary'])
        except:
            summary_dict[j] = [d['summary']]
print(json_list)
#dictionary 중복 제거
for i in range(len(summary_dict)):
    summary_dict[i] = list(set(summary_dict[i]))
    #print(len(summary_dict[i]))

data: 6: 100%|██████████| 4640/4640 [00:00<00:00, 779429.31it/s]

['.ipynb_checkpoints', 'abs_my_summary.json', 'my_summary_n_tonull.json', 'abs_3_my_summary.json', '1923_tonull.json', 'best.json', '1923_del_summary.json']


In [17]:
body = {
        "settings": {
            "analysis": {
                "filter": {
                    "my_shingle":{
                        "type":"shingle"
                    },
                    "my_stemmer":{
                        "type": "stemmer"
                    }

                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "decompound_mode": "mixed",
                        "filter": ["my_shingle","my_stemmer"],

                    },

                    
                },
                "similarity": {"my_similarity": {"type": "BM25"}}        }
        },
        "mappings": {
            "dynamic": "strict",
            "properties": {"document_text": {"type": "text", "analyzer": "nori_analyzer"}}    }}

In [18]:
# Elasticsearch index 생성
# ! curl -X PUT localhost:9200/_cluster/settings -H "Content-Type: application/json" -d '{ "persistent": { "cluster.max_shards_per_node": "10000" } }'
import argparse
import json
import warnings
import re

from elasticsearch import Elasticsearch
from tqdm import tqdm


warnings.filterwarnings("ignore")  # default: show warnings


"""
    elasticsearch를 실행하기 전에 본 파일을 먼저 실행시켜주세요!
    data가 변경되지 않았을 경우 실행하지 않아도 됩니다.
"""


def set_index_and_server(index_name):
    # Connect to elasticsearch
    config = {"host": "localhost", "port": 9200}
    es = Elasticsearch([config], timeout=30)
    #print("Ping Elasticsearch :", es.ping())

    # Make index
    if es.indices.exists(index_name):
        #print("Index already exists. Creating a new one after deleting it...")
        pass
        #es.indices.delete(index=index_name)
    else:   
        es.indices.create(index=index_name, body=body)
    #print("Index creation has been completed")

    return es


def insert_wiki_data(es, index_name):
    # Load wiki data
    wiki_articles = load_answer_data(index_name)

    # Inserting wiki data
    for i, rec in enumerate(wiki_articles):
        try:
            es.index(index=index_name, id=i, body=rec)
        except:
            print(f"Unable to load document {i}.")

    # Show and count data
    # sample_doc = es.get(index=index_name,id=1)
    # print(sample_doc)
    n_records = es.count(index=index_name)["count"]
    #print(f"Succesfully loaded {n_records} into {index_name}")

    return


def preprocess(text):
    specials = {'\n': '', '\u2009': ' ', '\u3000': ' ', '…': '...', '\u200b': ' ', '(’': '(', '(‘': '('}
    for s in specials:
        text = text.replace(s, specials[s])#re.sub(s, specials[s], text)
    return text


def load_answer_data(index_name):
    wiki_contexts = summary_dict[index_name]
    wiki_contexts = [preprocess(text) for text in wiki_contexts]
    wiki_articles = [
        {"document_text": wiki_contexts[i]} for i in range(len(wiki_contexts))
    ]

    return wiki_articles


def main(index_name):
    #print("Start to Set Elastic Search")
    es = set_index_and_server(index_name)
    insert_wiki_data(es=es, index_name=index_name)
    #print("Finish")


if __name__ == "__main__":
    for idx in tqdm(range(len(summary_dict))):
        index_name = idx
        main(index_name)

100%|██████████| 4640/4640 [24:42<00:00,  3.13it/s]


In [58]:
import time
import numpy as np
import pandas as pd
import argparse
from tqdm.auto import tqdm
from contextlib import contextmanager
from typing import List, Tuple, NoReturn, Any, Optional, Union

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from elasticsearch import Elasticsearch

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    #print(f"[{name}] done in {time.time() - t0:.3f} s")


def elastic_setting(index_name="origin-wiki-index"):
    config = {"host": "localhost", "port": 9200}
    es = Elasticsearch([config])
    #print("elastic serach ping :", es.ping())

    return es, index_name


def search_es(es, index_name, question_text, topk):
    # index: index to search, body: query to search
    query = {"query": {"match": {"document_text": question_text}}}
    res = es.search(index=index_name, body=query, size=topk)  # size: default 10, top k

    return res


class SparseRetrieval:
    def __init__(self,index_name) -> NoReturn:

        # run_elastic_search.py를 먼저 실행시켜야합니다. 처음 한 번이면 될 것입니다!
        self.es, self.index_name = elastic_setting(index_name=index_name)

        # 삽입된 문서 1개 확인(es 결과 확인)
        # print(self.es.get(index=self.index_name, id=1))

    def retrieve_ES(
        self, query_or_dataset: Union[str, Dataset], topk: Optional[int] = 1
    ) -> Union[Tuple[List, List], pd.DataFrame]:

        # Retrieve한 Passage를 pd.DataFrame으로 반환합니다.
        total = []
        with timer("query exhaustive search"):
            doc_scores, doc_indices, doc = self.get_relevant_doc_bulk_ES(
                query_or_dataset["question"], topk=topk
            )

        # 본 task에서 query는 1개이다.
        topK_context = ""
        for i, find_doc_idx in enumerate(doc):
            if find_doc_idx != []:
                topK_context = doc[i][0]["_source"]["document_text"]

        tmp = {
            # Query와 해당 id를 반환합니다.

            "question": query_or_dataset["question"],
            "context": topK_context,
        }
        total.append(tmp)

        cqas = pd.DataFrame(total)
        return cqas

    def get_relevant_doc_bulk_ES(
        self, queries: List, topk: Optional[int] = 1
    ) -> Tuple[List, List]:

        doc = []
        doc_scores = []
        doc_indices = []

        for question in queries:

            documents = search_es(self.es, self.index_name, question, topk)
            doc.append(documents["hits"]["hits"])

            doc_score = []
            doc_indice = []

            for hit in documents["hits"]["hits"]:
                doc_score.append(hit["_score"])
                doc_indice.append(hit["_id"])

            doc_scores.append(doc_score)
            doc_indices.append(doc_indice)

        return doc_scores, doc_indices, doc


if __name__ == "__main__":
    # query 생성
    # query를 doc_name으로 생성한다.
    full_ds = {}
    '''
    full_ds = {0:{question:질문}}
    '''
    regex = r'\([^)]*\)'
    for i,d in enumerate(data):
        d['Meta']['doc_name'] = re.sub(r'\([^)]*\)', '', d['Meta']['doc_name'])
        full_ds[i] = {'question':d['Meta']['doc_name']}
        
    # 새로운 df 정의
    new_df = pd.DataFrame(
        [], columns=["question","context"]
    )
    # 모든 index에 대해서 top1 뽑는다
    for index_name in tqdm(range(len(summary_dict))):
        retriever = SparseRetrieval(index_name)
        result_retriever = retriever.retrieve_ES(full_ds[index_name], topk=1)
        new_df = new_df.append(result_retriever,ignore_index=True)

  0%|          | 0/4640 [00:00<?, ?it/s]

In [59]:
new_df

,question,context
0,무령왕릉 청동거울 일괄,"공주시 무령왕릉에서 발견된 청동거울로 청동신수경, 의자손수대경, 수대경 3점이다. ..."
1,백자 대병,높이 47cm의 대형 백자 병으로 목이 유난히 길어 속칭 거위병이라고도 부른다. 긴...
2,무령왕비 금귀걸이,굵은 고리를 중심으로 작은 장식들을 연결하여 만든 것이 백제 때 귀고리 2쌍으로 길...
3,영월보덕사해우소,"해우소는 '근심을 해결하는 장소' 라는 뜻의 사찰에서 화장실을 이르는 말인데, 해우..."
4,완도신흥사목조약사여래좌상,완도군 완도읍 군내리 신흥사에 모셔진 약사여래좌상은 원래 해남 대흥사 소속암자인 심...
...,...,...
4635,201221 한국전쟁 전후 민간인 희생자 위령시설 국제설계공모 당선작 발표.hwp,한국전쟁 전후 민간인 희생자 위령시설을 건립하기 위한 국제설계 공모에서 SGHS 설...
4636,201228 2021년 해양수산 분야 이렇게 달라집니다.hwp,국적선사는 2021년 긴급한 화물 수요가 있는 항로를 중심으로 상대적으로 취약한 중...
4637,201127 중대본회의 보도자료.hwp,정부지방자치단체 지시로 지시로 병상을 비웠으나 환자 치료에 사용하지 못한 병상 손실...
4638,7.9 신소재식품과.hwp,"식품청과 농촌진흥청은 7월 9일 고단백 식품으로 탄수화물, 지방, 단백질 등 3대 ..."


In [60]:
with open(path_dir+json_list[4],"r") as f:
    best_data = json.load(f)

with open('test_summary.json', encoding='utf-8') as file:
    test_dataset = json.load(file)
    
for j in range(len(new_df)):
    if new_df['context'][j]=='':
        new_df['context'][j] = best_data[j]['summary']

for j in range(len(new_df)):
    if new_df['context'][j]=='':
        print(j)

In [61]:
summary_list = list(new_df['context'])
print(len(summary_list))

4640


In [62]:
for i in range(len(summary_list)):
    test_dataset[i]['summary'] = summary_list[i]

In [63]:
test_dataset[0]

{'original': '공주시 무령왕릉에서 발견된 청동거울로 청동신수경, 의자손수대경, 수대경 3점이다. 청동신수경은 ‘방격규구문경’이라는 중국 후한의 거울을 모방하여 만든 것이다. 거울 내부에는 반나체 인물상과 글이 새겨져 있는데 이는 한나라의 거울에서 흔히 볼 수 있는 것이다. 의자손수대경은 중국 한대의 수대경을 본떠 만든 방제경이다. 거울 중앙의 꼭지를 중심으로 9개의 돌기가 있고, 안에는 크고 작은 원과 7개의 돌기가 솟아있다. 내부 주위의 테두리에는 명문이 새겨져 있으나 선명하지 못하여 알아볼 수 없다. 수대경 역시 한나라 때 동물 문양을 새겨 넣은 수대경을 본떠서 만들어진 방제경이다. 그러나 한나라 거울에 비해 선이 굵고 무늬가 정교하지 못하다.',
 'summary': '공주시 무령왕릉에서 발견된 청동거울로 청동신수경, 의자손수대경, 수대경 3점이다. 청동신수경은 ‘방격규구문경’이라는 중국 후한의 거울을 모방하여 만든 것이다. 수대경 역시 한나라 때 동물 문양을 새겨 넣은 수대경을 본떠서 만들어진 방제경이다.',
 'Meta': {'passage_id': 'REPORT-cultural_assets-00164-01180',
  'doc_name': '무령왕릉 청동거울 일괄 (武寧王陵 銅鏡 一括)',
  'category': 'cul_ass',
  'author': None,
  'publisher': None,
  'publisher_year': None,
  'doc_origin': '문화재청'}}

In [64]:
with open('elastic_del_b.json', 'w', encoding="utf-8") as file:
    json.dump(test_dataset, file, indent='\t', ensure_ascii=False)